In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.a2c import A2C
from networks.a2c.actor import Actor
from networks.a2c.critic import Critic

import matplotlib.pyplot as plt
import torch

norm_cols = ['Volume XRP']
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 30
n_features = 4

env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-01-01',
        'end_date': '2021-02-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)

actor_model = Actor(
    state_dim=batch_dur*n_features+3,
    action_dim=3,
)
critic_model = Critic(state_dim=batch_dur*n_features+3)
a2c = A2C(
    env=env, 
    actor=actor_model,
    critic=critic_model,
    n_actns=3,
    actor_optmz=torch.optim.Adam(actor_model.parameters(), lr=0.0008),
    critic_optmz=torch.optim.Adam(critic_model.parameters(), lr=0.0005),
    hyprprms={
        'gamma': 0.9995,
    },
    log_freq=1,
    mdl_pth='../models/a2c_nn_3',
    load_models=False,
)

In [ ]:
a2c.run(1000)


Ep: 0 | TS: 59970 | L: 0.848 | R: -16.96 | P: 17.41 | R.Avg P: 17.41 | NW: 117.41 | R.Avg NW: 117.41 | R.U: 214
Ep: 1 | TS: 119940 | L: 0.487 | R: -35.27 | P: -5.28 | R.Avg P: 6.06 | NW: 94.72 | R.Avg NW: 106.06 | R.U: 159
Ep: 2 | TS: 179910 | L: 0.639 | R: -24.9 | P: 3.64 | R.Avg P: 5.26 | NW: 103.64 | R.Avg NW: 105.26 | R.U: 161
Ep: 3 | TS: 239880 | L: 0.962 | R: -10.5 | P: 15.67 | R.Avg P: 7.86 | NW: 115.67 | R.Avg NW: 107.86 | R.U: 177
Ep: 4 | TS: 299850 | L: 1.101 | R: 0.19 | P: 31.92 | R.Avg P: 12.67 | NW: 131.92 | R.Avg NW: 112.67 | R.U: 199
Ep: 5 | TS: 359820 | L: 1.214 | R: 16.78 | P: 67.93 | R.Avg P: 21.88 | NW: 167.93 | R.Avg NW: 121.88 | R.U: 234
Ep: 6 | TS: 419790 | L: 1.275 | R: 17.32 | P: 59.15 | R.Avg P: 27.21 | NW: 159.15 | R.Avg NW: 127.21 | R.U: 257
Ep: 7 | TS: 479760 | L: 1.358 | R: 14.07 | P: 56.85 | R.Avg P: 30.91 | NW: 156.85 | R.Avg NW: 130.91 | R.U: 278
Ep: 8 | TS: 539730 | L: 1.402 | R: 24.39 | P: 94.13 | R.Avg P: 37.94 | NW: 194.13 | R.Avg NW: 137.94 | R.U: 

In [ ]:
rewards, profits, actions = a2c.evaluate(start_dt='2021-01-30 15:15', duration=500, show_pred=True)

In [ ]:
rewards, profits, actions = a2c.evaluate(start_dt='2021-01-31 15:15', duration=500, show_pred=True)

## Visualizations

### Training 

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in a2c.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

In [ ]:
# torch.save(a2c.actor.state_dict(), '../models/a2c_nn_3/actor')
# torch.save(a2c.critic.state_dict(), '../models/a2c_nn_3/critic')

In [ ]:
# import pickle
# with open("../pickles/a2c_logs_main.pickle","wb") as f:
#     temp = dict(a2c.logs)
#     pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)